In [30]:
import openai
import textwrap as _tw
from credentials import api_key
openai.api_key = api_key

MODEL = "gpt-3.5-turbo"

MODEL4 = 'gpt-4'

Verify working


In [32]:


initial_messages = [{"role":"system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": "Why is the sky blue?"},
                    {"role": "assistant", "content" : "The sky is not really blue, but "}]
completion = openai.ChatCompletion.create(
    model=MODEL,
    messages= initial_messages)
print(_tw.fill(completion.choices[0].message.content, width=100))

appears that way because of the way our atmosphere scatters sunlight. When light from the sun enters
our atmosphere, it encounters gases and particles in the air that scatter the light in all different
directions. This happens most strongly with shorter, blue wavelengths of light, which is why the sky
looks blue to us.


## Goals

* Generate a story and then interact with it by interjecting changes to it. 

*  TODO: Allow user to save/load a story.

* TODO: Add a method to create a story summary for continuation. (partially done, and seems to work). But Ideally we should have another class that manages the various 'chapters' and can combine them (without the 'story so far' part.)

In [35]:
initial_messages = [{"role":"system", "content": "You are a helpful editor."},
                    {"role": "user", "content": "Finish the final sentance in this text: The car was very fast! However, we"} ]
completion = openai.ChatCompletion.create(
    model=MODEL,
    messages= initial_messages)
print(_tw.fill(completion.choices[0].message.content, width=100))

were unable to appreciate its full potential due to the heavy traffic on the road.


In [67]:
class gpt_story:
    def __init__(self, topic = "anything", model = "gpt-4", max_tokens = 100):
        self.model = model
        self.topic = topic
        self.max_tokens = max_tokens
        self.based_on_summary = False
        self.messages = [{"role":"system", "content": "You are a creative writer."},
                                 {"role": "user", "content": f"Write a story about {topic}."}]


    def query_gpt(self):
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages= self.messages, max_tokens = self.max_tokens)
        self.messages = self.messages + [dict(completion.choices[0].message)]

    def tell_story(self):
        self.query_gpt()
        self.clean_up_last_message()
        return self
    
    # This serves to make sure the last message is a complete sentence. Currently if the last sentence is complete
    # it still adds another. This is a bug that should be fixed but not critical.
    def clean_up_last_message(self):
        last_message = self.messages[-1]['content']
        cleanup_messages = [{"role":"system", "content": "You are a helpful editor."},
                    {"role": "user", "content": "Finish the final sentence in this text if needed: " + last_message} ]
        completion = openai.ChatCompletion.create(model=MODEL, messages= cleanup_messages)
        self.messages[-1]['content'] = last_message + " " + completion.choices[0].message.content
        return self

        
    def continue_with(self, message):
        message_dict = {'role':'assistant', 'content' : message}
        self.messages = self.messages +  [message_dict]
        self.tell_story()
        return self


    def summarize(self): 
        story_so_far = ' '.join([message['content'] for message in self.messages[2:]])
        summary = openai.Completion.create(
            model = "text-davinci-003",
            prompt = story_so_far + "tl;dr",
            max_tokens = 100
        )
        # Generate a new story starting with the summary
        new_story = gpt_story(topic=self.topic, model=self.model, max_tokens=self.max_tokens)
        new_story.messages += [{"role": "user", "content": "The story so far: "+summary.choices[0].text}]
        new_story.based_on_summary = True
        return new_story
            

    def __repr__(self):
        if self.based_on_summary:
            return _tw.fill(' '.join([message['content'] for message in self.messages[3:]]), width=120)
        return _tw.fill(' '.join([message['content'] for message in self.messages[2:]]), width=120)
    
    def __len__(self):
        return len(self.messages)
    
    def __getitem__(self, key):
        return self.messages[key]
    
    def __iter__(self):
        return iter(self.messages)
 

For demonstration purposes, we will use a short max token length of 60. This will allow us to see the story generation process in a reasonable amount of time.

In [68]:
test = gpt_story("sailing", max_tokens=60, model=MODEL)
test.tell_story()

Once upon a time, there was a young man named Jack who dreamed of sailing the seas. He had a passion for the open water
and longed to feel the wind in his hair and the salty mist on his face. Jack had grown up near the ocean and had always
been fascinated by the stories that his father would tell him of faraway lands and exotic creatures that could only be
found in the vastness of the sea.

Ok, we can continue with the story generation process by just calling `tell_story` again and again:

In [70]:
test.tell_story()

Once upon a time, there was a young man named Jack who dreamed of sailing the seas. He had a passion for the open water
and longed to feel the wind in his hair and the salty mist on his face. Jack had grown up near the ocean and had always
been fascinated by the stories that his father would tell him of faraway lands and exotic creatures that could only be
found in the vastness of the sea. One day, Jack's dream finally came true. He had saved enough money to purchase a small
sailboat and set off on his adventure. The boat was old and weathered, but it was all he needed to start his journey.
With supplies and provisions on board, Jack hoisted the sails and let the wind guide him.  Days turned into weeks as he
sailed across the ocean. He faced many challenges along the way, from unexpected storms to broken equipment, but he
persevered, learning important lessons about navigation and survival along the way. As he sailed, he met other sailors
and made new friends , but he always kept his u

But, if we want to continue the story in a different way, we can add our own text to the story and then continue from there:

In [71]:
test.continue_with("One day Jack sailed into a small harbor on a tropical island.")

Once upon a time, there was a young man named Jack who dreamed of sailing the seas. He had a passion for the open water
and longed to feel the wind in his hair and the salty mist on his face. Jack had grown up near the ocean and had always
been fascinated by the stories that his father would tell him of faraway lands and exotic creatures that could only be
found in the vastness of the sea. One day, Jack's dream finally came true. He had saved enough money to purchase a small
sailboat and set off on his adventure. The boat was old and weathered, but it was all he needed to start his journey.
With supplies and provisions on board, Jack hoisted the sails and let the wind guide him.  Days turned into weeks as he
sailed across the ocean. He faced many challenges along the way, from unexpected storms to broken equipment, but he
persevered, learning important lessons about navigation and survival along the way. As he sailed, he met other sailors
and made new friends , but he always kept his u

Finally, if the story starts getting too long, we can summarize it and then continue from there:

In [72]:
summary = test.summarize()

In [73]:
summary

The story so far:  A young man, Jack, saved up to buy a sailboat and set off on an adventure to explore the seas. He
faced many challenges along his journey, making new friends and learning important lessons, until he eventually reached
a small harbor on a tropical island. His journey was all about the experiences he had along the way.

In [75]:
summary.tell_story()

The story so far:  A young man, Jack, saved up to buy a sailboat and set off on an adventure to explore the seas. He
faced many challenges along his journey, making new friends and learning important lessons, until he eventually reached
a small harbor on a tropical island. His journey was all about the experiences he had along the way. When Jack sailed
into the harbor of the small island, he was taken aback by its beauty. The water was crystal clear and bright blue, and
the sand on the shore was as white as snow. He decided to anchor his sailboat and swim to shore to explore the little
island.  As he walked along the beach, he noticed the lush vegetation and colorful flowers that surrounded him, making
him feel like he had stepped into a tropical paradise.